In [6]:
# predefinitions
import pandas as pd
directory="altman z_all(sheet1-2).xlsx"
fail_condition = 1.8 #must be equal or below this number
naValues=["#N/A N/A", "#N/A Review"]

In [11]:
def getCompanyHistory(filename, sheet, skipRows, naValues=None):
    """
    Generates a Data Frame from a table of altman z values of companies
    
    :param filename: an Excel file (.xlsx) containing a table of companies and
                    their altmen z values according to dates
    :type filename: str
    :param sheet: Name of the sheet to extract from the file
    :type sheet: str
    :param skipRows: a list of indexes, indicating which rows to skip from the
                    top of the file
    :type skipRows: list of int >= 0
    :param naValues: values in the given table that are to be considered "NaN"
    :type naValues: scalar, str, list-like, or dict, default None
    
    :returns: A Data Frame containing altman z values of companies
    :rtype: pandas.DataFrame
    
    Example:
    >> getCompanyHistory("altman z_all(sheet1-2).xlsx","Sheet1", [0,1,2,4,5],
                             ["#N/A N/A", "#N/A Review"])
    """
    # extracting data
    cols = pd.read_excel(filename, encoding="latin-1", skiprows = skipRows,
                         na_values = naValues, sheet_name = sheet,
                         header=None, nrows=1, index_col = 0).values[0]
    data = pd.read_excel(filename, encoding="latin-1",
                           skiprows = skipRows, na_values = naValues,
                           sheet_name = sheet, index_col = 0)
    
    # modificatons on the extracted data
    data.columns = cols # rename columns
    data=data.dropna(axis=1,how="all") # drop columns with no values
    return data.loc[:,~data.columns.duplicated()] # drop duplicated columns

In [12]:
def weedSuccessful(dataFrame, failCond):
    """
    Weeds out companies that are considered not to have "failed" in the past
    from a given data frame of companies and their altman z values throughout
    the years
    
    :param dataFrame: 
    
    :param failcond: a company which has ever had an altman z value less than
                    or equal to this number will be considered to have "failed"
                    in the past
    :type failcond: float
    
    :returns: A Data Frame containing altman z values of  companies that have
                "failed" in the past
    :rtype: pandas.DataFrame
    """
    # weeding out companies that don't fail
    data_lower = dataFrame.where(dataFrame<=failCond)
    data_lower = data_lower.dropna(axis=1,how="all")
    columns_to_keep=list(data_lower.columns)
    return dataFrame.filter(columns_to_keep,axis=1)

In [13]:
def getCompanyFailHistory(filename, sheet, failCond, skipRows, naValues=None):
    """
    Generates a Data Frame from a table of altman z values of companies
    Weeds out companies that are considered not to have "failed" in the past
    
    :param filename: an Excel file (.xlsx) containing a table of companies and
                    their altmen z values according to dates
    :type filename: str
    :param sheet: Name of the sheet to extract from the file
    :type sheet: str
    :param failcond: a company which has ever had an alttman z value less than
                    or equal to this number will be considered to have "failed"
                    in the past
    :type failcond: float
    :param skipRows: a list of indexes, indicating which rows to skip from the
                    top of the file
    :type skipRows: list of int >= 0
    :param naValues: values in the given table that are to be considered "NaN"
    :type naValues: scalar, str, list-like, or dict, default None
    
    :returns: A Data Frame containing altman z values of  companies that have
                "failed" in the past
    :rtype: pandas.DataFrame
    
    Example:
    >> getCompanyFailHistory("altman z_all(sheet1-2).xlsx","Sheet1", 1.8, [0,1,2,4,5],
                             ["#N/A N/A", "#N/A Review"])
    """
    data=getCompanyHistory(filename, sheet, skipRows, naValues)
    data=weedSuccessful(data,failCond)
    return data